In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from data_preprocessing.ipynb import get_labels

ModuleNotFoundError: No module named 'data_preprocessing'

# Logistic Regression
Logistic Regression ist eine Variante der linearen Regression, die für binäre Klassifikationen verwendet wird.

Einlesen des Trainingsdatensatzes und des Testdatensatzes.

In [ ]:
def read_data():
    train_set = pd.read_csv('data/train_set.csv')
    test_set = pd.read_csv('data/test_set.csv')
    return train_set, test_set

train_set, test_set = read_data()

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_set, train_labels)
